Here we attempt to take a demographic score across postcodes (D_pc in the range [0,dmax]) and map it to a target range (T_pc in [tmin,tmax]) but also has the property that its weighted sum over postcodes gives a particular value for that state (T_state), i.e., if the set of weights for volume weighting over postcodes are f_pc st. sum_pc[f_pc] == 1, then sum_pc[(f_pc * T_pc)] == T_state 

In [1]:
import numpy as np
from scipy.optimize import fsolve
import plotly.graph_objects as go

In [2]:
sales = np.array([100, 200, 500, 250, 500, 6000, 5000, 20000]) # dummy sales data
f = sales / sum(sales)
D = np.array([0, 12, 47, 520, 62, 33, 400, 900]) # dummy demographic score
T_state = 0.5 # state level target
tmin = 0.25
tmax = 0.75

In [3]:
X = (D - min(D)) / (max(D) - min(D)) # normalize demographic score to [0, 1]

In [4]:
def Y(z,x):
    m = z[0]
    b = z[1]
    return m * x + b # this is a linear function by which we will transform the demographic normalized score

def T(z):
    XY = X * Y(z,X)
    return (XY - min(XY)) / (max(XY) - min(XY)) * (tmax - tmin) + tmin

In [5]:
def eq1(z):
    res = sum(f * T(z)) - T_state 
    return res

def eq2(z):
    m = z[0]
    b = z[1]
    return m * max(X) + b - 1

def my_function(z):
    return eq1(z), eq2(z)

In [6]:
z_guess = [-10,11]
z, infodict, ier, mesg = fsolve(my_function, z_guess, full_output=True)
print(f'Solved for m={z[0]}, b={z[1]}')
print(f'Solver return code: {ier}')
print(f'Solver mesg: {mesg}')
T_with_solve = T(z)
T_without_solve = T([0,1])
print(f'Residual for eq1: {eq1(z)}')
print(f'Residual for eq2: {eq2(z)}')
print(f'Residual for eq1(m=0,b=1): {eq1([0,1])}')
print(f'Residual for eq2(m=0,b=1): {eq2([0,1])}')

Solved for m=-5.511688305550781, b=6.511688305550781
Solver return code: 1
Solver mesg: The solution converged.
Residual for eq1: 0.0
Residual for eq2: 0.0
Residual for eq1(m=0,b=1): 0.0979245605052057
Residual for eq2(m=0,b=1): 0.0


In [7]:
print(D)
print(X)
print(T_with_solve)
print(T_without_solve)
print(sorted(T_with_solve))
print(sorted(T_without_solve))
print(sum(f*T_with_solve))
print(sum(f*T_without_solve))

[  0  12  47 520  62  33 400 900]
[0.         0.01333333 0.05222222 0.57777778 0.06888889 0.03666667
 0.44444444 1.        ]
[0.25       0.27232744 0.33453775 0.75       0.35987193 0.31017396
 0.71956799 0.5100973 ]
[0.25       0.25666667 0.27611111 0.53888889 0.28444444 0.26833333
 0.47222222 0.75      ]
[0.25, 0.2723274425172636, 0.31017396367143985, 0.3345377532532901, 0.35987193052648425, 0.5100972966919799, 0.7195679907476844, 0.75]
[0.25, 0.25666666666666665, 0.2683333333333333, 0.2761111111111111, 0.28444444444444444, 0.4722222222222222, 0.5388888888888889, 0.75]
0.5
0.5979245605052057


In [8]:
# plot up the linear modifier function

import plotly.graph_objects as go
x = np.linspace(0, 1, 100)
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=Y(z,x),
                    mode='lines',
                    name='Solved for modifier function'))
fig.add_trace(go.Scatter(x=x, y=Y(z_guess,x),
                    mode='lines',
                    name='Initial guess for modifer function'))

fig.show()

In [14]:
# plot the transformed demographic score 

import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=T(z), y=X*0,
                    mode='markers',
                    marker = dict(size=30),
                    name=f'Transformed with linear modifer function ({sum(f*T_with_solve)})'))
fig.add_trace(go.Scatter(x=T([0,1]), y=X*0,
                    mode='markers',
                    marker = dict(size=10),
                    name=f'Transformed without linear modifer function ({sum(f*T_without_solve)})'))

fig.show()